In [1]:
from openvino.runtime import Core
import os
from PIL import Image
import cv2
from sklearn.model_selection import train_test_split
from sklearn import svm,metrics,preprocessing
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.decomposition import PCA
#from sklearn.pipeline import Pipeline
#from sklearn.metrics.pairwise import pairwise_distances
from torchvision import transforms

from collections import defaultdict
import os
import random
import numpy as np
from tqdm import tqdm
import time
import pickle
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# путь к видео
DATA_DIR=r'D:\Users\amira\Documents\datasets\emotions\sample'


In [4]:
video_path=os.path.normpath(os.path.join(DATA_DIR,'video.mp4'))
print(video_path)
faces_path=os.path.normpath(os.path.join(DATA_DIR,'faces'))


D:\Users\amira\Documents\datasets\emotions\sample\video.mp4


In [5]:
FRAMES_TO_SKIP = 1

In [6]:
import facial_analysis.FacialImageProcessing as FacialImageProcessing
imgProcessing=FacialImageProcessing(False)

In [ ]:
if not os.path.exists(faces_path):
    os.mkdir(faces_path)

cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print('total_frames:',total_frames)

frame_count = 0
counter=0

for frame_count in tqdm(range(0, total_frames-1, FRAMES_TO_SKIP)):
    ret, frame_bgr = cap.read()
    counter+=FRAMES_TO_SKIP
    cap.set(cv2.CAP_PROP_POS_FRAMES, counter)
    if not ret:
        #cap.release()
        #break
        continue
    frame = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
    bounding_boxes, _ = imgProcessing.detect_faces(frame)
    if len(bounding_boxes)!=0:
        if len(bounding_boxes)>1:
            bounding_boxes=bounding_boxes[:1]

        b=[int(bi) for bi in bounding_boxes[0]]
        x1,y1,x2,y2=b[0:4]
        face_img=frame_bgr[y1:y2,x1:x2,:]        

        outfile=os.path.join(faces_path, str(counter)+'.png') 

        if np.prod(face_img.shape)==0:
            print('Empty face ',b,' found for ',filename)
            continue
        cv2.imwrite(outfile, face_img) 
        
    
cap.release()    

In [16]:
ie = Core()
# path to xml file with model converted to OpenVino's IR format. bin file should be in the same directory
model = ie.read_model(model=os.path.normpath(r"models\enet_b0_FP16\enet_b0_8_FP16.xml"))
# adding extra output layer
model.add_outputs(['652'])
feature_extractor = ie.compile_model(model=model, device_name="CPU")
input_layer = next(iter(feature_extractor.inputs))
output_layer = feature_extractor.outputs[1]
print(input_layer)
print(output_layer)

<ConstOutput: names[x.1] shape{1,3,224,224} type: f32>
<ConstOutput: names[652] shape{1,1280} type: f32>


In [19]:
test_transforms = transforms.Compose(
    [
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
    ]
)

def torch_transforms(filepath):
    face_img = Image.open(filepath)
    img_tensor = test_transforms(face_img)
    img_arr = img_tensor.numpy()
    img_arr = np.expand_dims(img_arr, axis=0)
    return img_arr

In [43]:
emotion_to_index = {'Angry':0, 'Disgust':1, 'Fear':2, 'Happy':3, 'Neutral':4, 'Sad':5, 'Surprise':6}


def get_features(data_dir):
    filename2features={}

    video_scores=[]
    X_isface=[]
    for img_name in os.listdir(data_dir):
        filepath=os.path.join(data_dir,img_name)
        input_image = torch_transforms(filepath)
        X_isface.append('noface' not in img_name)
            
        if input_image.size:
            scores = feature_extractor(inputs=[input_image])[output_layer]
            scores = scores[0].squeeze()
            video_scores.append(scores)
    filename2features['video_name']=(np.array(video_scores),np.array(X_isface))
    return filename2features

filename2features=get_features(faces_path)

In [48]:
def create_dataset(filename2features):
    x = []
    y = []
    has_faces=[]
    ind=0
    fn='video_name'

    features=filename2features[fn]
    total_features=None
    #print(len(features))
    if True:
        if len(features[0])!=0:
            cur_features=features[0][features[-1]==1]
        #print(prev,features.shape)
    else:
        cur_features=features[0]
    if len(cur_features)==0:
        has_faces.append(0)
        total_features=np.zeros_like(feature)
    else:
        has_faces.append(1)
        mean_features = (np.mean(cur_features, axis=0))
        std_features = (np.std(cur_features, axis=0))
        max_features = (np.max(cur_features, axis=0))
        min_features = (np.min(cur_features, axis=0))

        # join several features together
        feature = np.concatenate((mean_features, std_features, min_features, max_features), axis=None)                    

        total_features=feature
    
    if total_features is not None:
        x.append(total_features)
    x=np.array(x)
    has_faces=np.array(has_faces)
    print(x.shape)
    return x,has_faces

x_test, has_faces_test = create_dataset(filename2features)

(1, 5120)


In [49]:
x_test_norm=preprocessing.normalize(x_test,norm='l2')

In [52]:
 # load the model from disk
filename = 'linear_svc.sav'
loaded_model = pickle.load(open(filename, 'rb'))
y_pred = loaded_model.predict(x_test_norm)

In [54]:
y_pred

array([3])